This example assumes that you have an [Ollama](https://github.com/ollama/ollama) model running (with `ollama serve`) on some host that is reachable (whether a `localhost` of your computer or some other host with a public IP).
As a predictor, we use [txai_omics_3](https://github.com/SermanVS/txai_omics_3).

In [1]:
from llamba_framework.chatmodels.ollama import OllamaModel

In [2]:
chatbot = OllamaModel(url="http://127.0.0.1:11434/", endpoint="api/generate", model="llama3", num_threads=1, check_connection_timeout=15, request_timeout=15)
connection = chatbot.check_connection()
print(connection)

True


### Get a bioage model

In [3]:
from pathlib import Path
import os
import shap
import numpy as np
import pickle
import torch
from txai_omics_3.models.tabular.widedeep.ft_transformer import WDFTTransformerModel, FN_CHECKPOINT, FN_SHAP, TRAIN_DATA_PATH
from llamba.bioage_model import BioAgeModel

# Model
fn_model = FN_CHECKPOINT
model = WDFTTransformerModel.load_from_checkpoint(checkpoint_path=fn_model)
bioage_model = BioAgeModel(model=model)

# SHAP
fn_shap = FN_SHAP

def predict_func(x):
    batch = {
        'all': torch.from_numpy(np.float32(x)),
        'continuous': torch.from_numpy(np.float32(x)),
        'categorical': torch.from_numpy(np.int32(x[:, []])),
    }
    return model(batch).cpu().detach().numpy()
with open(fn_shap, 'rb') as handle:
    shap_dict = pickle.load(handle)

Lightning automatically upgraded your loaded checkpoint from v1.6.4 to v2.1.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint H:\Lobachevsky\txai_omics_3\data\immuno\model.ckpt`
h:\Lobachevsky\llamba\llamba_env\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: Metric `SpearmanCorrcoef` will save all targets and predictions in the buffer. For large datasets, this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


### Load data

In [4]:
import pandas as pd

my_data = {'CXCL9': 2599.629474, 
           'CCL22': 820.306524, 
           'IL6': 0.846377, 
           'PDGFB': 13400.666359, 
           'CD40LG': 1853.847406, 
           'IL27': 1128.886982,
           'VEGFA': 153.574220,
           'CSF1': 239.627236,
           'PDGFA': 1005.844290,
           'CXCL10': 228.229829,
           'Age': 90.454972 }

my_df = pd.DataFrame(my_data, index=[0])

# Model data

train_data = pd.read_excel(TRAIN_DATA_PATH)

### Initialize a connector

In [5]:
from llamba.connector import LlambaConnector

connector = LlambaConnector(bioage_model=bioage_model, chat_model=chatbot)

### Run an analysis task

In [6]:
res = connector.analyze(data=my_df, shap_dict=shap_dict)
print(res['analysis'])

Your bioage is 79 and your aging acceleration is -11, which means you are ageing slower than normal.

Here is some more information about your data. 

CXCL9: 2599.629474
CXCL9 (Chemokine C-X-C motif ligand 9) is a cytokine involved in immune responses, particularly in the process of T-cell activation and migration. An increased level of CXCL9 in older adults or individuals with age-related diseases may indicate:

1. Inflammaging: A low-grade chronic inflammatory state commonly seen in aging.
2. Immune senescence: Impaired T-cell function and reduced immune response capacity.
3. Disease progression: Elevated levels of CXCL9 have been linked to various conditions, such as cancer, atherosclerosis, and autoimmune disorders.

In the context of gerontology, increased CXCL9 levels may contribute to age-related decline in health and increase the risk of chronic diseases.

CD40LG: 1853.847406
CD40LG (CD154) is a receptor protein that plays a crucial role in T-cell activation and immune response